In [104]:
import re
import pandas as pd
import numpy as np
import psycopg2.extras as extras
import psycopg2

In [105]:
chess_data = open("chess.pgn").read()

In [106]:
#regex
date_regex = "(?<=\[Date \")(.*)(?=\"])"
white_regex = "(?<=\[White \")(.*)(?=\"])"
black_regex = "(?<=\[Black \")(.*)(?=\"])"
result_score_regex = "(?<=\[Result \")(.*)(?=\"])"
match_time_regex = "(?<=\[TimeControl \")(.*)(?=\"])"
end_result = "(?<=\[Termination \")(.*)(?=\"])"

In [107]:
dates = re.findall(date_regex, chess_data)
white_player = re.findall(white_regex, chess_data)
black_player = re.findall(black_regex, chess_data)
result_score = re.findall(result_score_regex, chess_data)
match_time_and_inc = re.findall(match_time_regex, chess_data)
end_result = re.findall(end_result, chess_data)

if_white_won = list(map(lambda s: s[0], result_score))
if_black_won = list(map(lambda s: s[2], result_score))

In [108]:
start_time_amount = list()
increments = list()
for time in match_time_and_inc:
    if "+" in time:
        start_time_amount.append(time[:-3])
        increments.append(time[-2:])
    else:
        start_time_amount.append(time)
        increments.append(None)
    

In [109]:
game_ended_by = list()

for end_string in end_result:
    if "time" in end_string:
        game_ended_by.append("Time")
    elif "checkmate" in end_string:
        game_ended_by.append("Checkmate")
    elif "game abandoned" in end_string:
        game_ended_by.append("Abandoned")
    elif "drawn by agreement" in end_string:
        game_ended_by.append("Drawn by Aggreement")
    elif "stalemate" in end_string:
        game_ended_by.append("Stalemate")
    elif "resignation" in end_string:
        game_ended_by.append("Resignation")
    else:
        game_ended_by.append(None)

In [110]:
start_time_amount_minutes = list(map(lambda n: int(n)//60, start_time_amount))
print(start_time_amount_minutes)

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 10, 10, 10, 10, 10, 10, 10, 10, 10, 15, 10, 10, 5, 10, 10, 10, 8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 5, 10, 10, 10, 10, 10, 20, 10, 15, 1, 15]


In [111]:
chess_final_data = pd.DataFrame(
    {
        'datePlayed': dates,
        'whitePlayer': white_player,
        'blackPlayer': black_player,
        'ifWhiteWon': if_white_won,
        'ifBlackWon': if_black_won,
        'startTimeSeconds': start_time_amount,
        'secondIncrements': increments,
        'startTimeMinutes': start_time_amount_minutes 
    }
)

In [112]:
chess_final_data[['ifWhiteWon', 'ifBlackWon', 'startTimeSeconds', 'secondIncrements', 'startTimeMinutes']] = chess_final_data[['ifWhiteWon', 'ifBlackWon', 'startTimeSeconds', 'secondIncrements', 'startTimeMinutes']].apply(pd.to_numeric)
chess_final_data['datePlayed'] = pd.to_datetime(chess_final_data['datePlayed'])
chess_final_data['secondIncrements'] = chess_final_data['secondIncrements'].fillna(0.0).astype(int)

In [113]:
conn_string = open("connectionString.txt").read()
def execute_values(conn, df, table):
  
    tuples = [tuple(x) for x in df.to_numpy()]
  
    cols = ','.join(list(df.columns))
    # SQL query to execute
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("the dataframe is inserted")
    cursor.close()
  
conn_string = open("connectionString.txt").read()
  
conn = psycopg2.connect(
   conn_string
)
  
execute_values(conn, chess_final_data, 'Chess')

the dataframe is inserted


In [114]:
"""
CREATE TABLE CHESS (
   datePlayed date, 
   whitePlayer varchar,
   blackPlayer varchar,
   ifWhiteWon int,
   ifBlackWon int,
   startTimeSeconds int,
   secondIncrements int,
   startTimeMinutes int
)

"""

'\nCREATE TABLE CHESS (\n   datePlayed date, \n   whitePlayer varchar,\n   blackPlayer varchar,\n   ifWhiteWon int,\n   ifBlackWon int,\n   startTimeSeconds int,\n   secondIncrements int,\n   startTimeMinutes int\n)\n\n'